In [0]:
# Databricks notebook source
import pandas as pd
from datetime import datetime
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType, IntegerType, StructType, StructField
from pyspark.sql.functions import to_date, year, month, day, hour, minute, when, avg, regexp_replace, mean, count, round
from pyspark.sql import SparkSession
from pyspark.sql.functions import date_format
from pyspark.sql.functions import col, regexp_extract, to_timestamp
from pyspark.sql.functions import unix_timestamp, col, expr

# Define Azure Blob Storage paths
raw_folder_path = "/mnt/public-transport-data/raw/"
processed_folder_path = "/mnt/public-transport-data/processed/"
csv_file_path = "/mnt/public-transport-data/processed_fils/"

# Define the schema for the CSV file
schema = StructType([
    StructField("name_file", StringType(), True),
    StructField("date", StringType(), True)
])

# Function to extract the month number from the filename
def extract_month(filename):
    parts = filename.split("_")
    for part in parts:
        if part.isdigit():
            return int(part)
    return 0  # Return 0 if month number is not found

# List to keep track of imported datasets
imported_datasets = []

# Load the existing dataset names from the CSV file
existing_datasets = set()
try:
    # Read existing dataset names from the CSV file
    existing_df = spark.read.option("header", True).schema(schema).csv(csv_file_path)
    existing_datasets = set(existing_df.select("name_file").rdd.flatMap(lambda x: x).collect())
except Exception as e:
    print("Error reading existing datasets:", str(e))

# Get a list of files in the 'raw' folder in Azure Blob Storage
files_in_raw_folder = dbutils.fs.ls(raw_folder_path)
data_files = [file.name for file in files_in_raw_folder if file.isFile() and file.name not in existing_datasets]

# Sort the data files by their numeric part (e.g., month)
data_files.sort(key=lambda x: extract_month(x))

# Function to import a new dataset
def import_new_dataset():
    if data_files:
        # Get the next dataset to import
        dataset_to_import = data_files.pop(0)
        
        # Check if dataset is already imported
        if dataset_to_import in existing_datasets:
            print(f"Dataset {dataset_to_import} already imported. Skipping...")
        else:
            # Update the list of imported datasets
            imported_datasets.append(dataset_to_import)
            
            # Save dataset name and current date to the CSV file
            save_to_csv(dataset_to_import, datetime.now().strftime('%Y-%m-%d'))
            
            # Display the dataset (use dataset_to_import)
            print(f"Displaying dataset: {dataset_to_import}")
            dataset_df = spark.read.option("header", True).csv(raw_folder_path + dataset_to_import)
            
            def convert_to_24_hour(time_str):
                return to_timestamp(time_str, 'h:mm:ss a').cast('string')

            # Apply the function to DepartureTime and ArrivalTime columns
            dataset_df = (dataset_df.withColumn('DepartureTime', regexp_extract(col('DepartureTime'), r'(\d+:\d+:\d+ [APM]+)',1))
                          .withColumn('ArrivalTime', regexp_extract(col('ArrivalTime'), r'(\d+:\d+:\d+ [APM]+)', 1)))

            # Convert DepartureTime and ArrivalTime to 24-hour format
            dataset_df = (dataset_df.withColumn('DepartureTime', convert_to_24_hour(col('DepartureTime')))
                         .withColumn('ArrivalTime', convert_to_24_hour(col('ArrivalTime'))))
            
            dataset_df = (dataset_df.withColumn('DepartureTime', date_format(col('DepartureTime'), 'HH:mm:ss'))
                         .withColumn('ArrivalTime', date_format(col('ArrivalTime'), 'HH:mm:ss')))
            
            # Calculate the duration in seconds
            dataset_df = dataset_df.withColumn('DurationSeconds', 
                                                (unix_timestamp(col('ArrivalTime'), 'HH:mm:ss') - 
                                                unix_timestamp(col('DepartureTime'), 'HH:mm:ss')))

            # Convert the duration from seconds to HH:mm:ss format
            dataset_df = dataset_df.withColumn('Duration', 
                                                expr("from_unixtime(DurationSeconds, 'HH:mm:ss')"))
            
            dataset_df = dataset_df.withColumn('DelayCategory',
                                        when(col('Delay') == 0, 'Pas de Retard')
                                        .when((col('Delay') >= 1) & (col('Delay') <= 10), 'Retard Court')
                                        .when((col('Delay') >= 11) & (col('Delay') <= 20), 'Retard Moyen')
                                        .when(col('Delay') > 20, 'Long Retard')
                                        .otherwise('Unknown'))
            

            peak_passenger_threshold = 70

            # Identify peak and off-peak hours based on the number of passengers
            dataset_df = (dataset_df.withColumn('Hour', hour(col('DepartureTime')))
                                      .withColumn('HourCategory',
                                                  when(col('Passengers') >= peak_passenger_threshold, 'Peak Hour')
                                                  .otherwise('Off-Peak Hour')))



            
            dataset_df.show()  # Show the contents of the dataset
            

            
            # Copy the dataset to the 'processed' folder
            copy_to_processed(dataset_to_import, dataset_df)
            
            # TODO: Import the dataset (use dataset_to_import)
            # Replace the following print statement with your dataset import logic
            print(f"Imported dataset: {dataset_to_import}")
    else:
        print("No more datasets to import.")

# Function to save dataset name and date to the CSV file
def save_to_csv(dataset_name, import_date):
    try:
        # Create a DataFrame with the new dataset import record
        new_record_df = spark.createDataFrame([(dataset_name, import_date)], schema)
        
        # Read existing dataset names and dates from the CSV file
        existing_df = spark.read.option("header", True).schema(schema).csv(csv_file_path)
        
        # Union the existing dataset with the new record
        updated_df = existing_df.union(new_record_df)
        
        # Write the updated DataFrame to the CSV file
        updated_df.write.option("header", True).mode("overwrite").csv(csv_file_path)
    except Exception as e:
        print("Error saving to CSV:", str(e))

def copy_to_processed(dataset_name, dataset_df):
    destination_path = processed_folder_path + dataset_name
    try:
        # Write the dataset_df to the 'processed' folder
        dataset_df.write.option("header", True).mode("overwrite").csv(destination_path)
        print(f"Dataset {dataset_name} copied to 'processed' folder.")
    except Exception as e:
        print(f"Failed to copy dataset {dataset_name} to 'processed' folder. Error: {str(e)}")
    
import_new_dataset()
import_new_dataset()
